In [25]:
import os
import pickle
import pandas as pd
import torch

from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, BertConfig

In [3]:
dataset_dir = "/opt/ml/input/data/train/train.tsv"
dataset = pd.read_csv(dataset_dir, delimiter="\t", header=None)
dataset.head()

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님


In [6]:
with open('/opt/ml/input/data/label_type.pkl', 'rb') as f:
    label_type = pickle.load(f)  # dictionary

In [7]:
label = []
for i in dataset[8]:
    if i == "blind":
        label.append(100)
    else:
        label.append(label_type[i])
out_dataset = pd.DataFrame({'sentence':dataset[1], 'entity_01':dataset[2], 'entity_02':dataset[5], 'label':label,})

out_dataset.head()

,sentence,entity_01,entity_02,label
0,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,자동차,17
1,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,27석,0
2,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,UEFA,6
3,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,공격수,2
4,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,퍼쿤 씨 인트라팃,8


In [14]:
def tokenized_dataset(dataset, tokenizer):
    concat_entity = []
    for e01, e02 in zip(dataset['entity_01'], dataset['entity_02']):
        temp = ''
        temp = e01 + '[SEP]' + e02
        concat_entity.append(temp)
    tokenized_sentences = tokenizer(
        concat_entity,
        list(dataset['sentence']),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=100,
        add_special_tokens=True,
    )

    return tokenized_sentences

In [15]:
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [16]:
tokenizer.decode(tokenizer.encode("랜드로버[SEP]자동차", out_dataset.loc[0, "sentence"]))

'[CLS] 랜드로버 [SEP] 자동차 [SEP] 영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버 ( Land Rover ) 와 지프 ( Jeep ) 가 있으며, 이 브랜드들은 자동차의 종류를 일컫는 말로 사용되기도 한다. [SEP]'

In [19]:
print(tokenizer.encode("랜드로버[SEP]자동차", out_dataset.loc[0, "sentence"], max_length=100, padding='max_length'))

[101, 9167, 15001, 11261, 41605, 102, 9651, 99183, 102, 50266, 11489, 9405, 24974, 24683, 9477, 90578, 9625, 119376, 12692, 45725, 9651, 99183, 10459, 9376, 42771, 70186, 9167, 15001, 11261, 41605, 113, 12001, 57836, 114, 9590, 9706, 28396, 113, 13796, 19986, 114, 8843, 22634, 117, 9638, 9376, 42771, 22879, 9651, 99183, 10459, 9684, 46520, 11513, 9641, 119298, 11018, 9251, 11261, 9405, 24974, 118800, 27792, 16139, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [24]:
tokenized_train = tokenized_dataset(out_dataset, tokenizer)
tokenized_train

{'input_ids': tensor([[   101,   9167,  15001,  ...,      0,      0,      0],
        [   101,   9311,  16323,  ...,      0,      0,      0],
        [   101,  68495,  37905,  ...,      0,      0,      0],
        ...,
        [   101,   9328, 118782,  ...,    123, 101322,    102],
        [   101,  49780,  16617,  ...,      0,      0,      0],
        [   101,   9730,  30858,  ...,      0,      0,      0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [26]:
from torch.utils.data import Dataset

class RE_Dataset(Dataset):
    def __init__(self, tokenized_dataset, labels):
        self.tokenized_dataset = tokenized_dataset
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.tokenized_dataset.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [32]:
out_label = out_dataset['label'].values
RE_train_dataset = RE_Dataset(tokenized_train, out_label)

In [36]:
next(iter(RE_train_dataset))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


{'input_ids': tensor([   101,   9167,  15001,  11261,  41605,    102,   9651,  99183,    102,
          50266,  11489,   9405,  24974,  24683,   9477,  90578,   9625, 119376,
          12692,  45725,   9651,  99183,  10459,   9376,  42771,  70186,   9167,
          15001,  11261,  41605,    113,  12001,  57836,    114,   9590,   9706,
          28396,    113,  13796,  19986,    114,   8843,  22634,    117,   9638,
           9376,  42771,  22879,   9651,  99183,  10459,   9684,  46520,  11513,
           9641, 119298,  11018,   9251,  11261,   9405,  24974, 118800,  27792,
          16139,    119,    102,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [37]:
# RE_train_data는 transformer에서 제공하는 Trainer 객체의 Input으로 사용